# Monte-Carlo Simulation

## Aufgabenstellung

Wir betrachten weiter den Federspeicher aus Assignment 1. Diesmal simulieren wir die Ergebnisse der Fertigung für $N=10.000$ Zylinder. In diesem Assignment kommt die Federrate dazu. Das Federpaket besteht aus 24 Einzelfedern, die ineinander gesteckt auf dem Umfang des Zylinder verteilt sind. Die Toleranzen der Federn sind recht groß. Die Federn sind nominal

- $c_1 = (100 \pm 20)\, \mathrm{N/mm}$ 
- $c_2 = (150 \pm 30)\, \mathrm{N/mm}$ 
- $c_3 = (200 \pm 40)\, \mathrm{N/mm}$ 

Die Gesamtfederrate ergibt sich als

$$
c = \sum_{i=1}^{3} \sum_{j=1}^{8} c_{i,j}
$$

Die geometrischen Eigenschaften entspricht dem ersten Assignment:

 - Deviation $d$/mm: Gleichverteilt, $[-0.05, 0.05]\,\mathrm{mm}$
 - Deviation $h_{1}$/mm: Gleichverteilt, $[-0.05, 0.05]\,\mathrm{mm}$
 - Deviation $h_{2}$/mm: Gleichverteilt, $[-0.05, 0.05]\,\mathrm{mm}$
 - Dev. $l$/mm: Gleichverteilt, $[-0.05, 0.05]\,\mathrm{mm}$

Mit diesen Abweichungen vom Mittelwert ergibt sich die Kraft des Federspeichers als

$$
F = 2 i c(1 + h_1 + h_2 + l + d)
$$

mit der Übersetzungsrate des Zylinders $i = 2.6$.

Fragestellungen:

1. Federpaket:
   - a. Wie verhält sich das Federpaket im nominalen Zustand? Simuliert dazu mindestens $N=10^4$ stochastisch unabhängige Einzelfedern und bestimmt die Federrate von aus diesen Federn zusammengesetzten Federpaketen.
   - b. Die Federn $c_3$ liegen innerhalb der Toleranzen mit $c_3 < 200 \, \mathrm{N/mm}$. Wie ändert sich die Federrate?
2. Federspeicherzylinder: Welche Verteilungen ergeben sich für die Federspeicherzylinder mit Federpakete nach a. bzw. b.? Wie entwickelt sich der Anteil der konformen Einheiten ($F_{min} = 18 \,\mathrm{kN}$)?

Beantwortet die Fragestellungen in einem technischen Bericht mit Einleitung, Analyse und Beantwortung der Fragen.

# Assigment 3 Monte-Carlo-Simulation 
### a.Nominaler Zustand

In [34]:
import numpy as np
import matplotlib.pyplot as plt

N = 10000

Zu beginn erzeugen wir  $N=10^4$ stochastisch unabhängige Einzelfedern mit der Numpy.random Funktion

In [35]:
def Simulation_Einzelfedern(N):
    c1_values = np.random.uniform(80, 120, N)
    c2_values = np.random.uniform(120, 180, N)
    c3_values = np.random.uniform(160, 240, N)

    return c1_values, c2_values, c3_values

Nun werden die Simulierten Federn getestet, indem die Funktion mit dem Wert N=1 aufgerufen wird.  

In [36]:
Simulation_Einzelfedern(1)


(array([105.46681219]), array([157.39102741]), array([205.1181458]))

Zu erkennen ist das jeweiles drei einzelne Federn generiert werden, welche einen Zufälligen Wert im Toleranz Bereich erhalten. 

Nun wird eine Funktion erstellt, womit die Federrate ausgerechnet wird. Hierbei handelt es sich 

In [37]:
def Gesamtfederrate(N):
    c1_values, c2_values, c3_values = Simulation_Einzelfedern(N)

    # Berechnung der Gesamtfederrate c
    total_federrate = np.sum(np.sum([c1_values, c2_values, c3_values]))

    return total_federrate

In [45]:
gesamtfederrate_beispiel = Gesamtfederrate(N)
print(f"Gesamtfederrate für {N} Einzelfedern: {int(gesamtfederrate_beispiel)} N/mm")

Gesamtfederrate für 10000 Einzelfedern: 4501768 N/mm
